In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import openpyxl

In [2]:
URL="https://www.themoviedb.org/movie/top-rated"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}
response=requests.get(URL,headers=headers,timeout=50)
page=response.text

In [3]:
soup=bs(page,'lxml')

In [4]:
# link='https://www.themoviedb.org/'+ soup.find(name='div',class_='card style_1').find(name='a',class_='image').get('href')
# link

In [5]:
all_links=[]
for i in range(1,15):
  URL=f'https://www.themoviedb.org/movie/top-rated?page={i}'
  response=requests.get(URL,headers=headers)
  page=response.text
  soup=bs(page, 'lxml')
  divs = soup.find_all(name='div', class_='card style_1')
  for div in divs:
    link = 'https://www.themoviedb.org/' + div.find(name='a', class_='image').get('href')
    all_links.append(link)
  # link='https://www.themoviedb.org/'+ soup.find_all(name='div',class_='card style_1').find(name='a',class_='image').get('href')
  # all_links.append(link)

In [6]:
URL=all_links[0]
response=requests.get(URL,headers=headers)
page=response.text
soup=bs(page, 'lxml')

In [7]:
name=soup.find(name='h2').text.strip()

In [8]:
release_date=soup.find('span',class_='release').text.strip()
# release_date

In [9]:
genres=soup.find('span',class_='genres').text.replace('\xa0', '').strip()
# genres

In [10]:
runtime=soup.find('span',class_='runtime').text.strip()
# runtime

In [11]:
percent=soup.find('span', class_='icon')
user_score=(" ".join(percent.get('class')) .split()[-1].split('-')[-1].replace('r',"")) + " %"
user_score

'87 %'

In [12]:
tagline=soup.find('h3',class_='tagline').text
tagline

'Fear can hold you prisoner. Hope can set you free.'

In [13]:
description=soup.find('div',class_='overview').text.strip()
# description

In [14]:
profiles=soup.find_all('li', class_='profile')
person_info = []
for person in profiles:
  name = soup.find('li', class_='profile')
  name=person.find('p').text
  tag=person.find('p',class_='character').text
  person_info.append({
    'name': name,
    'title': tag
  })
# person_info 
# # profiles

For multiple movies

In [15]:
movie_details=[]
for URL in all_links:
  headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
  } 
  # print(URL)
  response=requests.get(URL,headers=headers)
  page=response.text
  soup=bs(page, 'lxml')
  # movvie details
  movie_name=soup.find(name='h2').text.strip()
  release_date=soup.find('span',class_='release').text.strip()
  genres=soup.find('span',class_='genres').text.replace('\xa0', '').strip()
  runtime=soup.find('span',class_='runtime').text.strip()
  percent=soup.find('span', class_='icon')
  user_score=(" ".join(percent.get('class')) .split()[-1].split('-')[-1].replace('r',"")) + " %"
  tagline_element=soup.find('h3',class_='tagline')
  tagline = tagline_element.text.strip() if tagline_element else "-"
  description=soup.find('div',class_='overview').text.strip()
  profiles=soup.find_all('li', class_='profile')
  person_info = []
  for person in profiles:
    name = soup.find('li', class_='profile')
    name=person.find('p').text
    tag=person.find('p',class_='character').text
    person_info.append({
      'name': name,
      'title': tag
    })
  movie_details.append({
        'movie_name': movie_name,
        'movie_link': URL,
        'release_date': release_date,
        'genres': genres,
        'runtime': runtime,
        'user_score': user_score,
        'tagline': tagline,
        'description': description,
        'profiles': person_info
      })
    

In [16]:
movie_details[0]

{'movie_name': 'The Shawshank Redemption\n(1994)',
 'movie_link': 'https://www.themoviedb.org//movie/278-the-shawshank-redemption',
 'release_date': '09/23/1994 (US)',
 'genres': 'Drama,Crime',
 'runtime': '2h 22m',
 'user_score': '87 %',
 'tagline': 'Fear can hold you prisoner. Hope can set you free.',
 'description': 'Imprisoned in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.',
 'profiles': [{'name': 'Frank Darabont', 'title': 'Director, Screenplay'},
  {'name': 'Stephen King', 'title': 'Novel'}]}

In [17]:
movie_details_cleaned=movie_details
# len(movie_details_cleaned)
for movie in movie_details_cleaned:
    # Join profile names and titles into a single string for each profile
    # print(movie)
    profiles_info = []
    for profile in movie['profiles']:
        # print(profile)
        profiles_info.append(f"{profile['name']} ({profile['title']})")
    
    # Create a single string from the profiles
    movie['profiles'] = '; '.join(profiles_info)

# Example output
print(movie_details_cleaned[15])


{'movie_name': 'David Attenborough: A Life on Our Planet\n(2020)', 'movie_link': 'https://www.themoviedb.org//movie/664280-david-attenborough-a-life-on-our-planet', 'release_date': '10/04/2020 (US)', 'genres': 'Documentary,Drama', 'runtime': '1h 23m', 'user_score': '85 %', 'tagline': '-', 'description': 'The story of life on our planet by the man who has seen more of the natural world than any other. In more than 90 years, Attenborough has visited every continent on the globe, exploring the wild places of our planet and documenting the living world in all its variety and wonder. Addressing the biggest challenges facing life on our planet, the film offers a powerful message of hope for future generations.', 'profiles': 'Keith Scholey (Director); Alastair Fothergill (Director); Jonathan Hughes (Director)'}


In [ ]:
# movie_details_cleaned[:3]
df=pd.DataFrame(movie_details_cleaned)
pd.set_option('display.max_colwidth', None)

In [53]:
df.head(1)['profiles']

0    Frank Darabont (Director, Screenplay); Stephen King (Novel)
Name: profiles, dtype: object

In [55]:
import numpy as np

# Initialize the columns with NaN values
df['director'] = np.nan
df['screenplay'] = np.nan
df['novel'] = np.nan
df['writer'] = np.nan
df['story'] = np.nan
df['characters'] = np.nan

def extract_roles(profiles, roles):
    # Create a dictionary to hold the roles and their respective names
    matches = {role: [] for role in roles}  
    for part in profiles.split(';'):
        part = part.strip()
        for role in roles:
            if role in part:
                # Get the name by removing the role part in parentheses
                name = part.split('(')[0].strip()
                matches[role].append(name)
    
    # Return matches as a formatted string for each role
    return {role: ', '.join(set(names)) if names else np.nan for role, names in matches.items()}

# Define the roles we want to extract
roles = ['Director', 'Screenplay', 'Novel', 'Writer', 'Story', 'Characters']

# Apply the extraction function to each row in the DataFrame
for index, row in df.iterrows():
    results = extract_roles(row['profiles'], roles)
    df.at[index, 'director'] = results['Director']
    df.at[index, 'screenplay'] = results['Screenplay']
    df.at[index, 'novel'] = results['Novel']
    df.at[index, 'writer'] = results['Writer']
    df.at[index, 'story'] = results['Story']
    df.at[index, 'characters'] = results['Characters']

# Display specific columns for the first 5 rows
print(df[['profiles', 'director', 'screenplay', 'novel', 'writer', 'story', 'characters']].tail(5))


                                                                                                              profiles  \
275                                                            Florian Henckel von Donnersmarck (Director, Screenplay)   
276                                                                Miloš Forman (Director); Peter Shaffer (Screenplay)   
277                                                                 George Roy Hill (Director); David S. Ward (Writer)   
278                                                                                Jeanne Herry (Director, Screenplay)   
279  Noé Santillán-López (Director); Jose Manuel Flandes (Story); José Alberto López (Story); Angélica Gudiño (Writer)   

                             director                        screenplay novel  \
275  Florian Henckel von Donnersmarck  Florian Henckel von Donnersmarck   NaN   
276                      Miloš Forman                     Peter Shaffer   NaN   
277                   Ge

C:\Users\VARUNI\AppData\Local\Temp\ipykernel_4964\2734701558.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Frank Darabont' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'director'] = results['Director']
C:\Users\VARUNI\AppData\Local\Temp\ipykernel_4964\2734701558.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Frank Darabont' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'screenplay'] = results['Screenplay']
C:\Users\VARUNI\AppData\Local\Temp\ipykernel_4964\2734701558.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Stephen King' has dtype incompatible with float64, please explicitly cast to a compatible dtype f

In [ ]:
pd.set_option('display.max_colwidth', 100)
df['movie_name'] = df['movie_name'].str.strip()
df['movie_name'] = df['movie_name'].str.replace('\n', '', regex=False)
df = df.fillna('-')

In [61]:
df.to_excel('moviesdb.xlsx',index=False)